In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import netCDF4
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras import preprocessing
from tqdm import tqdm
import cartopy.crs as ccrs
import gc

2022-12-15 14:01:34.607864: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/cray/pe/papi/6.0.0.12/lib64:/opt/cray/job/2.2.4-7.0.3.1_3.24__g36b56f4.ari/lib64:/opt/intel/compilers_and_libraries_2020.2.254/linux/compiler/lib/intel64:/opt/intel/compilers_and_libraries_2020.2.254/linux/mkl/lib/intel64:/global/common/software/nersc/cori-2022q1/sw/darshan/3.4.0/lib
2022-12-15 14:01:34.607895: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Data Preprocessing

In [2]:
cd '/global/cscratch1/sd/aeide'

/global/cscratch1/sd/aeide


In [3]:
#Opens netcdf file with zonal MSLP at 40s and 65s, in 6-hourly increments
zonal_means = xr.open_dataset('./era5/zonal_mean_slp.nc')
zonal_means

<xarray.Dataset>
Dimensions:  (time: 61364)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 ... 2020-12-31T18:00:00
Data variables:
    avg40s   (time) float32 ...
    avg65s   (time) float32 ...

In [4]:
#averages each latitude over time
mean40s = zonal_means.avg40s.mean('time')
mean65s = zonal_means.avg65s.mean('time')

In [5]:
#computes (centered) SAM, resamples to daily from 6-hourly, normalizes by standard deviation
SAM = (zonal_means.avg40s - mean40s) - (zonal_means.avg65s - mean65s)
SAM = SAM.resample(time = 'D').mean('time')
SAM = SAM / SAM.std('time')

In [6]:
def autocorrelation(series, tau):
    n = len(series)
    acc = 0
    for i in tqdm(range(n - 1 - tau)):
        acc += series[i]*series[i + tau]
    return acc / (n-tau)

In [7]:
#smooths by taking 2w rolling average
smoothSAM = SAM.rolling(time = 14, min_periods = 1, center = True).mean()

In [9]:
autocorrelation(smoothSAM, 7)

100%|██████████| 15333/15333 [00:20<00:00, 757.76it/s]


<xarray.DataArray ()>
array(0.51976091)

In [ ]:
#example plot

date = '1999-04-20'
time_span = [np.datetime64(date) + np.timedelta64(i, 'D') for i in range(360)]

plt.figure()
SAM.sel(time = time_span).plot(color = 'blue')
smoothSAM.sel(time = time_span).plot(color = 'red')
plt.title('SAM: daily and 1w rolling mean')
plt.show()

In [ ]:
#opens netcdf files containing long-run means and std's, respectively, of SLP in each grid cell
#(quarter-degree grid)

means = xr.open_dataset('./slp_means.nc').means
stdevs = xr.open_dataset('./slp_std.nc').stdevs

# Data Generation

In [ ]:
class DataGenerator(utils.Sequence):
    
    def __init__(self, list_IDs, outputs, backward_window_length = 4, 
                 batch_size = 16, lat_min = -80.0, lat_max = -20.0, 
                 shuffle = True):
        
        #list_IDs: list dates of form 'yyyy-mm-dd' in datetime64 format
        #outputs: predictand (smoothSAM)
        #backward_window_length: number of weeks to use in the input
        #lat_min, lat_max: where to crop the grid

        
        self.lat_min = lat_min
        self.lat_max = lat_max
        self.lat_range = np.arange(self.lat_min, self.lat_max, 0.25)
        self.mu = means.sel(latitude = self.lat_range)
        self.sigma = stdevs.sel(latitude = self.lat_range)
        self.batch_size = batch_size
        self.outputs = outputs
        self.backward_window_length = backward_window_length
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        # returns number of batches per epoch
        return int(np.floor(len(self.list_IDs) / self.batch_size))
        
    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        
        X, y = self.__data_generation(list_IDs_temp)
        
        return X, y
        
    def on_epoch_end(self):
        # Updates indices after each epoch'
        self.indices = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indices)
            
    def __data_generation(self, list_IDs_temp):
        
        #input X has size (batch_size, 4*(lat_max - lat_min), 4 * 360, backward_window_length)
        X = np.empty((self.batch_size, int(4 * (self.lat_max - self.lat_min)), 1440, self.backward_window_length))
        
        #output y has lenght (batch_size)
        y = np.empty(self.batch_size)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            
            #date at which to retrieve output--- the average of SAM over next 2 weeks equals smoothSAM(t + 7)
            foreward_ID = ID + np.timedelta64(14, 'D')
            #blah blah blah need to do hap aldskfaneian 
    
            # Store inputs
            ds = xr.open_dataset('./era5_daily_with_lag/mslp.' + str(ID) + '.nc').MSL.sel(time_bins = [t for t in range(4 - self.backward_window_length, 4)])
            # The 'daily_with_lag' datasets contain weekly averaged MSLP for 4 consecutive weeks
            # The week ending in the day listed in the file name corresponds to the time_bin = 4 index in the ds
            ds = ( ds.sel(latitude = self.lat_range) - self.mu ) / self.sigma
            X[i,] = np.moveaxis(ds.to_numpy(), 0, -1)

            # Store outputs
            
            y[i] = float(self.outputs.sel(time = foreward_ID))
            
        return X, y

# Metrics

In [ ]:
class PatternCorrelation(keras.metrics.Metric):
    
    #computes running pattern correlation coefficients
    
    def __init__(self, batch_size, name = 'pattern_cor'):
        super(PatternCorrelation, self).__init__(name = name)
        
        self.batch_size = batch_size
        
        self.count = self.add_weight(name = 'count', initializer = 'zeros')
        
        self.running_total_true = self.add_weight(name = 'running_true', initializer = 'zeros')
        self.running_total_pred = self.add_weight(name = 'running_pred', initializer = 'zeros')
        
        self.running_square_total_true = self.add_weight(name = 'running_square_true', initializer = 'zeros')
        self.running_square_total_pred = self.add_weight(name = 'running_square_pred', initializer = 'zeros')
        
        self.running_dot_product = self.add_weight(name = 'running_dot_product', initializer = 'zeros')
        
    def update_state(self, y_true, y_pred, sample_weight = None):
        
        self.count.assign_add(self.batch_size)
        
        self.running_total_true.assign_add(tf.math.reduce_sum(y_true, axis = 0)[0])
        self.running_total_pred.assign_add(tf.math.reduce_sum(y_pred, axis = 0)[0])
        
        self.running_square_total_true.assign_add(tf.math.reduce_sum(tf.math.square(y_true), axis = 0)[0])
        self.running_square_total_pred.assign_add(tf.math.reduce_sum(tf.math.square(y_pred), axis = 0)[0])
        
        self.running_dot_product.assign_add(tf.math.reduce_sum(tf.math.multiply(y_true, y_pred), axis = 0)[0])
        
    def result(self):
        
        joint_second_moment = self.running_dot_product / self.count 
        
        first_moment_true = self.running_total_true / self.count
        first_moment_pred = self.running_total_pred / self.count
        
        second_moment_true = self.running_square_total_true / self.count
        second_moment_pred = self.running_square_total_pred / self.count
        
        std_true = tf.math.sqrt(second_moment_true - tf.math.square(first_moment_true))
        std_pred = tf.math.sqrt(second_moment_pred - tf.math.square(first_moment_pred))
        
        return ( joint_second_moment - tf.math.multiply(first_moment_true, first_moment_true) ) / tf.math.multiply(std_true, std_pred)
        
    def reset_state(self):
        for var in self.variables:
            var.assign(0)

In [ ]:
def PC(y1, y2):
    #pattern correlation of two series y1 and y2 of same length
    mu1 = np.mean(y1)
    mu2 = np.mean(y2)
    
    sig1 = np.std(y1)
    sig2 = np.std(y2)
    
    Y1 = (y1 - mu1) / sig1
    Y2 = (y2 - mu2) / sig2
    
    return np.mean( np.multiply(Y1, Y2) )

In [ ]:
def damped_persistence(series, tau):
    #damped persistence of series with lag tau
    
    n = len(series)
    AC1 = 0
    for i in range(len(series) - 1):
        AC1 += series[i] * series[i+1]
    
    damped_persistence_forecast = [0] * (tau - 1)
    
    for x in series:
        damped_persistence_forecast.append(x**((tau * AC1) / n))
    
    return damped_persistence_forecast    

# Model

In [ ]:
#set parameters for data generation
list_IDs = np.arange('1979-02', '2000-01', dtype = 'datetime64[D]')
backward_window_length = 2
batch_size = 16
lat_min = -75.0
lat_max = -25.0

lat_range = np.arange(lat_min, lat_max, 0.25)
mu = means.sel(latitude = lat_range)
sigma = stdevs.sel(latitude = lat_range)

In [ ]:
#the model
model = keras.Sequential([
            
            #make convolutional filters 3 x 3
    
            layers.Conv2D(28, (3, 3), padding = 'same',
                          batch_input_shape = (batch_size, int(4*(lat_max - lat_min)), 1440, backward_window_length),
                          use_bias = True, kernel_regularizer=keras.regularizers.l2(l=0.05)),
            layers.MaxPool2D(pool_size = (3, 3)),
            layers.Activation('relu'),
            
            layers.Conv2D(28, (3, 3), padding = 'same', use_bias = True, kernel_regularizer=keras.regularizers.l2(l=0.05)),
            layers.MaxPool2D(pool_size = (3, 3)),
            layers.Activation('relu'),
            
            layers.Conv2D(28, (3, 3), padding = 'same', use_bias = True, kernel_regularizer=keras.regularizers.l2(l=0.05)),
            layers.MaxPool2D(pool_size = (3, 3)), 
            layers.Activation('relu'),
    
            layers.Conv2D(28, (3, 3), padding = 'same', use_bias = True, kernel_regularizer=keras.regularizers.l2(l=0.05)),
            layers.MaxPool2D(pool_size = (3, 3)),
            layers.Activation('relu'),
            
            layers.Conv2D(28, (3, 3), padding = 'same', use_bias = True, kernel_regularizer=keras.regularizers.l2(l=0.05)),
            layers.Activation('relu'),
        
            layers.Flatten(),
    
            layers.Dense(100, kernel_regularizer=keras.regularizers.l2(l=0.05)),
            #Increase size here
            layers.Activation('relu'),
    
            layers.Dense(100, kernel_regularizer=keras.regularizers.l2(l=0.05)),
    
            layers.Dropout(0.2),
            
            layers.Dense(1, use_bias = True)
          ]
        )

In [ ]:
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics = [PatternCorrelation(batch_size = batch_size)])
#change learning rate in ADAM

In [ ]:
model.summary()

In [ ]:
gen = DataGenerator(list_IDs = list_IDs, outputs = smoothSAM,
                    backward_window_length = backward_window_length,
                    batch_size = batch_size, lat_min = lat_min,
                    lat_max = lat_max, shuffle = True)

In [ ]:
model.fit_generator(gen, epochs = 4, verbose = True)

# Save/Load Model

In [ ]:
model.save('./sam_predict4')

In [ ]:
#Need to fix the custom pattern correlation metric; tensorflow isn't able to save it as written
model = tf.keras.models.load_model('./sam_predict5', compile=False)

In [ ]:
model.summary()

In [ ]:
def damped_persistence(series, tau):
    
    n = len(series)
    AC1 = 0
    for i in range(len(series) - 1):
        AC1 += series[i] * series[i+1]
    
    damped_persistence_forecast = [0] * (tau - 1)
    
    for x in series:
        damped_persistence_forecast.append(x**((tau * AC1) / n))
    
    return damped_persistence_forecast    

# Evaluation

In [ ]:
#specify test dates and make test generator. parameters should be the same as what the model was trained on
test_IDs = np.arange('2000-01-01', '2015-12-28', dtype = 'datetime64[D]')
test_gen = DataGenerator(list_IDs = test_IDs, outputs = smoothSAM,
                    backward_window_length = backward_window_length,
                    batch_size = batch_size, lat_min = lat_min,
                    lat_max = lat_max, shuffle = True)

In [ ]:
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics = [PatternCorrelation(batch_size = batch_size)])

In [ ]:
model.evaluate_generator(test_gen, verbose = True)

# Visualization

In [ ]:
lat_range = np.arange(lat_min, lat_max, 0.25)
mu = means.sel(latitude = lat_range)
sigma = stdevs.sel(latitude = lat_range)

In [ ]:
int(4*(lat_max - lat_min))

In [ ]:
def ForecastVisualize(start_date, span):
    #plots the predicted series vs. true series for span days beginning with start_date
    
    dates = [np.datetime64(start_date) + np.timedelta64(i, 'D') for i in range(span)]
    plot_dates = [date + np.timedelta64(14, 'D') for date in dates]
    pred_outputs = []
    true_outputs = []
    
    for date in tqdm(dates):
    
        y_true = float(smoothSAM.sel(time = date + np.timedelta64(14, 'D')))
        #y_true = float(smoothSAM.sel(time = date))
    
        inp = xr.open_dataset('./era5_daily_with_lag/mslp.' + str(date) + '.nc').MSL.sel(time_bins = [t for t in range(4 - backward_window_length, 4)])
        inp = ( inp.sel(latitude = lat_range) - mu ) / sigma
        inp = np.moveaxis( inp.to_numpy(), 0, -1)
        inp = np.expand_dims(inp, 0)
    
        y_pred = float(model(inp)[0][0])
    
        true_outputs.append(y_true)
        pred_outputs.append(y_pred)
    
    fig = plt.figure()
    plt.plot(plot_dates, pred_outputs, color = 'red', label = 'Predicted')
    plt.plot(plot_dates, true_outputs, color = 'blue', label = 'True')
    plt.xticks(rotation = 45)
    plt.title('SAM ' + start_date + ' ' + str(span) + ' days')
    plt.legend(loc="upper left")
    plt.show()
    
    return fig

In [ ]:
fig = ForecastVisualize('2010-06-01', 365*3)

# XAI

In [ ]:
lat_range = np.arange(lat_min, lat_max, 0.25)
lon_range = np.arange(0, 360 , 40)

In [ ]:
def get_gradients(model, inp):
    # model: tf MODEL to compute gradients on
    # inp: tf VARIABLE to use as input
    
    with tf.GradientTape() as tape:
        tape.watch(inp)
        out = model(inp)[0][0]
    grads = tape.gradient(out, inp)
    return grads

In [ ]:
def get_smooth_gradients(model, inp, m):
    # model: tf MODEL to compute smooth gradients on
    # inp: tf VARIABLE to use as input
    # m: smoothing parameter
    
    inp_shape = tuple(inp.get_shape().as_list())
    stacked_inp = tf.repeat(inp, [m], axis = 0)
    tf.random.set_seed(6)
    perturbations = tf.random.normal(inp_shape)
    
    perturbed_inp = tf.add(stacked_inp, perturbations)
    
    grads = get_gradients(model, perturbed_inp)
    
    smooth_grads = tf.reshape(tf.math.reduce_mean(grads, axis = 0), inp_shape)
    
    return smooth_grads 

In [ ]:
def get_inp_dot_gradients(model, inp):
    # model: tf MODEL to compute smooth inp*grad on
    # inp: tf VARIABLE to use as input
    
    grads = get_gradients(model, inp)
    inp_dot_gradients = tf.math.multiply(inp, grads)
    return inp_dot_gradients

In [ ]:
def interpolate_inputs(inp, N):
    
    # utility function used in integrated gradients
    # interpolates from baseline (all 0s) to inp in regular increments
    
    inp_shape = tuple(inp.get_shape().as_list())
    baseline = tf.zeros(shape = inp_shape)
    alphas = tf.linspace(start = 0.0, stop = 1.0, num = N + 1)
    alphas_x = alphas[:, tf.newaxis, tf.newaxis, tf.newaxis]
    delta = inp - baseline
    interpolated_inputs = baseline + alphas_x * delta
    
    return interpolated_inputs

def get_integrated_gradients(model, inp, N):
    # model: tf MODEL to compute smooth integrated gradients on
    # inp: tf VARIABLE to use as input
    # N: number of terms in Riemann sum approximation of integral
    
    interpolated_inputs = interpolate_inputs(inp, N)
    grads = get_gradients(model, interpolated_inputs)
    pre_integrated_grads = tf.expand_dims(tf.math.reduce_mean(grads, axis = 0), axis = 0)
    integrated_grads = tf.math.multiply(inp, pre_integrated_grads)
    
    return integrated_grads

In [ ]:
def make_tf_input(inp_date, lat_range = lat_range):
    # utility function generating tf VARIABLE from inp_date and lat_range
    
    ds = xr.open_dataset('./era5_daily_with_lag/mslp.' + inp_date + '.nc').MSL.sel(time_bins = [t for t in range(4 - backward_window_length, 4)])
    inp = ( ds.sel(latitude = lat_range) - mu ) / sigma
    inp = np.moveaxis(inp.to_numpy(), 0, -1)
    inp = np.expand_dims(inp, 0)
    return tf.Variable(inp)

In [ ]:
def tensor_normalize(tensor):
    # utility function to normalize a tf tensor by max absolute value entry
    
    normalized = tensor / tf.reduce_max(tf.abs(tensor))
    return normalized

In [ ]:
def plot_saliency(saliency_map, inp, figsize, lag = 0, cmap = 'seismic'):
    
    #making the plot
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize = figsize)
    ax1.imshow(inp[0,:,:,3 - lag], cmap, origin = 'lower', extent = [0, 360, lat_min, lat_max])
    ax1.set_title('Model Input')
    ax1.set_xticks(lon_range)
    ax1.set_yticks(lat_range_plt)
    im = ax2.imshow(tensor_normalize(saliency_map[0,:,:,3 - lag]), cmap, origin = 'lower', extent = [0, 360, lat_min, lat_max], norm = mcolors.Normalize(vmin = -1, vmax =1))
    ax2.set_title('Saliency Map')
    ax2.set_xticks(lon_range)
    ax2.set_yticks(lat_range_plt)
    plt.plot((0, 360), (-40,-40), 'black')
    plt.plot((0, 360), (-65,-65), 'black')
    
    return fig

In [ ]:
def plot_saliency2(saliency_map, figsize, cmap = 'seismic', norm = mcolors.Normalize(vmin = -1, vmax =1), projection = ccrs.Robinson(), lat = lat_range, lon = np.arange(0, 360, 0.25)):
    # plots inputs and saliency maps in some projection 
    # colorbar needs to be fixed!
    
    bounds = [-1, 1]
    fig = plt.figure(figsize = figsize)
    ax = plt.axes(projection = projection)
    ax.set_extent([0, 359.75, lat_min, lat_max], crs=ccrs.PlateCarree())
    im = plt.contourf(lon, lat, saliency_map, 60, transform=ccrs.PlateCarree(), cmap = 'seismic', alpha = 0.75, norm = norm)
    gl = ax.gridlines(draw_labels= True, xlocs=[], ylocs=[-65,-40], color = 'black', linestyle = '--')
    gl.n_steps = 90
    ax.coastlines()
    #cax = fig.add_axes([0.92, 0.4, 0.025, 0.2])
    #fig.colorbar(im, cax=cax)
    plt.title('Gradient Saliency Map')
    plt.show()
    
    return(fig)

In [ ]:
input_date = '2000-12-25'
inp = make_tf_input(input_date)
grads = tensor_normalize(get_gradients(model, inp)[0,:,:,1])
#smooth_grads = tensor_normalize(get_smooth_gradients(model, inp, 30)[0,:,:,3])
#inp_dot_grads = tensor_normalize(get_inp_dot_gradients(model, inp)[0,:,:,3])
#integrated_grads = tensor_normalize(get_integrated_gradients(model, inp, 50)[0,:,:,3])

In [ ]:
fig1 = plot_saliency2(grads, figsize = (15, 15))

In [ ]:
grads2 = tensor_normalize(get_gradients(model, inp)[0,:,:,0])
#smooth_grads = tensor_normalize(get_smooth_gradients(model, inp, 30)[0,:,:,3])
#inp_dot_grads = tensor_normalize(get_inp_dot_gradients(model, inp)[0,:,:,3])
#integrated_grads = tensor_normalize(get_integrated_gradients(model, inp, 50)[0,:,:,3])

In [ ]:
fig2 = plot_saliency2(grads2, figsize = (15, 15))

In [ ]:
inp_date = '1990-07-01'
inp = make_tf_input(input_date)
grads = tensor_normalize(get_gradients(model, inp)[0,:,:,1])
fig = plot_saliency2(grads, figsize = (15, 15))

In [ ]:
grads2 = tensor_normalize(get_gradients(model, inp)[0,:,:,0])
fig2 = plot_saliency2(grads2, figsize = (15, 15))

In [ ]:
inp = tensor_normalize(inp[0,:,:,0])
fig3 = plot_saliency2(inp, figsize = (15, 15))

In [ ]:
alphas = tf.linspace(start = 0.0, stop = 1.0, num = 20)
alphas_x = alphas[:, tf.newaxis, tf.newaxis, tf.newaxis]

In [ ]:
delta = inp - baseline
interpolated_inputs = baseline + alphas_x * delta

In [ ]:
interpolated_inputs

In [ ]:
weights = {weight.name.split('/')[0]: weight for weight in model.trainable_weights
                        if 'bias' not in weight.name}

In [ ]:
activations = [layer.output for layer in model.layers]
activations = activations[::-1]

In [ ]:
layer_names = [layer.name for layer in model.layers]
layer_names = layer_names[::-1]

In [ ]:
layer_names

In [ ]:
def relprop_conv(a, w, r, name, strides = (1, 1), padding = 'same'):
    
    #a: vector of activations from layer l
    #w: array of weights from layer l
    #r: relevances of neurons at layer l + 1
    
    if name == 'conv2d':
        a = tf.ones_like(a)
    
    z = tf.nn.conv2d(a, w, strides, padding)
    s = r / z
    c = tf.compat.v1.nn.conv2d_backprop_input(tf.shape(a), w, s, strides, padding)
    
    return c * a

In [ ]:
def relprop_dense(a, w, r):
    
    #a: vector of activations from layer l
    #w: array of weights from layer l
    #r: relevances of neurons at layer l + 1
    
    z = tf.matmul(a, w)
    
    s = r / z
    
    c = tf.matmul(s, tf.transpose(w))
    
    return c * a

In [ ]:
def relprop_pool(a, r, pool_size = (), strides = (1, 1), padding = 'same'):